In [1]:
import numpy as np
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import PIL
import os
from numpy import linalg as LA
import pickle

In [2]:
model = tf.keras.models.load_model("facenet_keras.h5")
for layer in model.layers:
    layer.trainable=False

W0308 18:59:28.624626 4700659136 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [3]:
def get_face_array(img_path):
    image = PIL.Image.open(img_path)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = PIL.Image.fromarray(face)
    image = image.resize((160, 160))
    face_array = np.asarray(image)
    mean, std = face_array.mean(), face_array.std()
    face_array = (face_array - mean) / std
    return np.expand_dims(face_array, axis=0).astype("float32")

In [4]:
def get_face_embedding(face_ary, model):
    return model.predict(face_ary)

In [28]:
d1 = {}
dir_path = "/Users/zetong/Desktop/buddhas_eagle/"
for file in os.listdir("/Users/zetong/Desktop/buddhas_eagle/user_database/img_database1"):
    if "jpg" in file:
        idx = file.replace(".jpg", "")
        d1[idx] = get_face_embedding(get_face_array(dir_path + "user_database/img_database1/" + file), model)

d2 = {}
dir_path = "/Users/zetong/Desktop/buddhas_eagle/"
for file in os.listdir("/Users/zetong/Desktop/buddhas_eagle/user_database/img_database2"):
    if "jpg" in file:
        idx = file.replace(".jpg", "")
        d2[idx] = get_face_embedding(get_face_array(dir_path + "user_database/img_database2/" + file), model)

d3 = {}
dir_path = "/Users/zetong/Desktop/buddhas_eagle/"
for file in os.listdir("/Users/zetong/Desktop/buddhas_eagle/user_database/img_database3"):
    if "jpg" in file:
        idx = file.replace(".jpg", "")
        d3[idx] = get_face_embedding(get_face_array(dir_path + "user_database/img_database3/" + file), model)

In [31]:
d = {}
for key in d1.keys():
    d[key] = (d1[key] + d2[key] + d3[key]) / 3
with open('d.pickle', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
def search(dic, q):
    vals = {}
    for key in dic.keys():
        vals[key] = LA.norm(dic[key] - q)
    if vals[min(vals, key=vals.get)] >= 10:
        return -1
    else:
        return min(vals, key=vals.get)

In [18]:
qi = get_face_embedding(get_face_array("/Users/zetong/Desktop/amell.jpg"), model)
search(d, qi)

-1

In [43]:
users = {"1": "Elon Musk", "2": "Tulsi Gabbard", "3": "Emma Watson",
        "4": "Bruce Lee", "5": "Antony Starr", "6": "Zetong Qi", "NOT_REG": "NOT REGISTERED", "7": "Stephen Amell"}
with open('users.pickle', 'wb') as handle:
    pickle.dump(users, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
"1.jpg".replace(".jpg", "")

'1'

In [ ]:
with open('users.pickle', 'wb') as handle:
    pickle.dump(, handle, protocol=pickle.HIGHEST_PROTOCOL)